## Task 2. 

In [198]:
import pandas as pd
import numpy as np

In [244]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import transform

import pyproj
from functools import partial

In [200]:
user_df = pd.read_csv('user_coordinates.csv')

In [201]:
user_df

,user_id,loc_lat,loc_lon
0,1,55.737564,37.345186
1,2,56.234564,37.234590
2,3,55.234578,36.295745
3,4,55.764578,38.295745


In [202]:
plgn_df = pd.read_csv('place_zone_coordinates.csv')

In [203]:
plgn_df

,place_id,loc_lat,loc_lon,point_number
0,1,55.747022,37.787073,0
1,1,55.751713,37.784328,1
2,1,55.753878,37.777638,2
3,1,55.751031,37.779351,3
4,2,55.803885,37.458311,0
5,2,55.808677,37.464054,1
6,2,55.809763,37.461314,2
7,2,55.810840,37.458654,3


In [204]:
def make_plgn_dict(file):
    plgn_dict = {}  
    for row in file.iterrows():
        plgn_id = row[1][0]
        plgn_dict.setdefault(plgn_id, [])        
        plgn_dict[plgn_id].append((row[1][1], row[1][2]))
    return plgn_dict   

In [205]:
polygon_dict = make_plgn_dict(plgn_df)

In [245]:
project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj('+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +no_defs'))

/usr/local/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [207]:
def search(row):
    number_of_places_available = 0
    for value in polygon_dict.values():
        poly = Polygon(value)
        poly_g = transform(project, poly)
        if poly_g.contains(Point([row['loc_lat'], row['loc_lon']])) == True:
            number_of_places_available += 1
    return number_of_places_available  

In [128]:
user_df['number_of_places_available'] = user_df.apply(search, axis=1)

In [130]:
user_df[['user_id', 'number_of_places_available']]

,user_id,number_of_places_available
0,1,0
1,2,0
2,3,0
3,4,0


**Что непосредственно делает pyproj? Проецирует географические координаты на сферический Меркатор. Ниже в качестве примера взят первый полигон и первый юзер и посчитано расстояние между ними до использования pyproj и после.**

In [238]:
a = (([55.747021999999994, 37.787073],
  [55.751713, 37.784328],
  [55.753878, 37.777638],
  [55.751031000000005, 37.779351]))

In [239]:
poly = Polygon(a)

In [241]:
point = Point(55.737564, 37.345186)

In [242]:
poly.contains(point)
poly.distance(point)

0.4327596086743821

In [246]:
poly_g = transform(project, poly)
p1_g = transform(project, point)

In [248]:
poly_g.contains(p1_g) 
poly_g.distance(p1_g)

60757.01056610022

## Task 3.

Мы в Яндекс.Еде любим поиграть в покер и настольные игры. Поэтому нам важно, чтобы будущие коллеги были сильны в математической статистике и теории вероятностей (так интереснее играть))) Если серьезно, то в большинстве аналитических задач у нас возникает необходимость применять математический аппарат в том или ином виде. Проще всего его проверить через учебные игровые задачи, чтобы не погружать в сложный контекст бизнеса. Вы подбрасываете кубик (6 граней), после чего у вас есть две возможности:

Взять себе сумму  ,равнуювыпавшемунакубикечислу(выпало3–получаете3 ) Отклонить результат первого броска и подбросить кубик второй раз. После чего уже взять себе столько $, сколько выпало во второй раз (второй бросок отклонять уже нельзя) Какую стратегию нужно выбрать, чтобы максимизировать ожидаемую прибыль (при каких значениях первого броска нужно перебрасывать?) Почему эта стратегия максимизирует прибыль?

Чему равна ожидаемая прибыль при этой стратегии (математическое ожидание)?

Моделирование различных вариантов бросков.

In [14]:
# Расчет мат.ожидания при разовом броске

import numpy as np

trials = 100000    # Количество попыток
money = 0

for i in range(trials):
    random_score = np.random.randint(1, 7)    
    if random_score == 1:
        money += 1
    elif random_score == 2:
        money += 2
    elif random_score == 3:
        money += 3
    elif random_score == 4:
        money += 4
    elif random_score == 5:
        money += 5
    elif random_score == 6:
        money += 6
        
mean_gain = money/trials
print(mean_gain)

3.50686


In [15]:
# перебрасываем кубик при выпадении 1, 2, 3, 4, 5
# возможные варианты: 6 1 2 3 4 5 6
# мат ожидание 1/6(6)+5/6(3,5) = 1 + 2,92 = 3,92


trials = 100000000    # Количество попыток
money = 0

for i in range(trials):
    random_score = np.random.randint(1, 7)    
    if random_score in [1,2,3,4,5]:
        random_score = np.random.randint(1, 7) 
        if random_score == 1:
            money += 1
        elif random_score == 2:
            money += 2
        elif random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        elif random_score == 6:
                money += 6
    else:
        money += 6
       
        
mean_gain = money/trials
print(mean_gain)

3.916687


In [16]:
# перебрасываем кубик при выпадении 1, 2, 3, 4
# возможные варианты: 5 6 1 2 3 4 5 6
# мат ожидание 1/3(5,5)+2/3(3,5) = 1,83 + 2,33 = 4,16

trials = 10000000    # Количество попыток
money = 0

for i in range(trials):
    random_score = np.random.randint(1, 7)       
    if random_score in [1,2,3,4]:
        random_score = np.random.randint(1, 7)  
        if random_score == 1:
            money += 1
        elif random_score == 2:
            money += 2
        elif random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        elif random_score == 6:
                money += 6
    else:
        if random_score == 5:
            money += 5
        else:
            money += 6
       
        
mean_gain = money/trials
print(mean_gain)

4.1658322


In [17]:
# перебрасываем кубик при выпадении 1, 2, 3
# возможные варианты: 4 5 6 1 2 3 4 5 6
# мат ожидание 1/2(5)+1/2(3,5) = 2,5 + 1,75 = 4,25

trials = 10000000    # Количество попыток
money = 0

for i in range(trials):
    random_score = np.random.randint(1, 7)    
    if random_score in [1,2,3]:
        random_score = np.random.randint(1, 7) 
        if random_score == 1:
            money += 1
        elif random_score == 2:
            money += 2
        elif random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        elif random_score == 6:
                money += 6
    else:
        if random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        else:
            money += 6
       
        
mean_gain = money/trials
print(mean_gain)

4.249648


In [18]:
# перебрасываем кубик при выпадении 1, 2
# возможные варианты: 3 4 5 6 1 2 3 4 5 6
# мат ожидание 2/3(4,5)+1/3(3,5) = 3 + 1,17 = 4,17


trials = 100000    # Количество попыток
money = 0

for i in range(trials):
    random_score = np.random.randint(1, 7)    
    if random_score in [1,2]:
        random_score = np.random.randint(1, 7) 
        if random_score == 1:
            money += 1
        elif random_score == 2:
            money += 2
        elif random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        elif random_score == 6:
                money += 6
    else:
        if random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        else:
            money += 6
       
        
mean_gain = money/trials
print(mean_gain)

4.16303


In [19]:
# перебрасываем кубик при выпадении 1
# возможные варианты: 2 3 4 5 6 1 2 3 4 5 6
# мат ожидание 5/6(4)+1/6(3,5) = 3,33 + 0,58 = 3,92



trials = 100000    # Количество попыток
money = 0

for i in range(trials):
    random_score = np.random.randint(1, 7)    
    if random_score == 1:
        random_score = np.random.randint(1, 7) 
        if random_score == 1:
            money += 1
        elif random_score == 2:
            money += 2
        elif random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        elif random_score == 6:
                money += 6
    else:
        if random_score == 2:
            money += 2
        elif random_score == 3:
            money += 3
        elif random_score == 4:
            money += 4
        elif random_score == 5:
            money += 5
        else:
            money += 6
       
        
mean_gain = money/trials
print(mean_gain)

3.91615
